In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
from utils import *
%pylab inline --no-import-all

Populating the interactive namespace from numpy and matplotlib


In [3]:
filters_names = ["jump2-~foot1-small_volume-yz0-~opentop1-top1",'~jump1-jump2-foot1-foot2-~isnew-~small_capq-small_volume-~opentop1-2']

In [13]:
filter1 = getFilter('~opentop2-jump1-jump2-small_volume-yz0')
filter2 = getFilter('yz0-yz1-~opentop2-isnew-~small_volume')

In [5]:
filter3 = data_df["open1"] > data_df["high_limit1"] * 0.99

In [6]:
filter2 = minute & ~isnew & ~foot(2,1.0) & ~jump2 & foot(2,0.9) & minute2 & break_top & (data_df["open2"] < data_df["close1"] * 1.03) & (data_df["minute_low"] > data_df["close1"] * 0.97)

In [11]:
valid_df = data_df[filter1 & filter2]
# valid_df = valid_df[isnew]
valid_df = valid_df.sort_values("minute")
valid_df = valid_df.groupby("date").first()
result(valid_df)

Empty DataFrame
Columns: [mean, count]
Index: []
1.0 1 1.10027744748


/Users/sgcy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [14]:
valid_df = data_df[filter2]
valid_df = valid_df.sort_values("minute")
valid_df = valid_df.groupby("date").first()
result(valid_df)

            mean  count
recent                 
3.0     0.993182      7
4.0     1.012833     17
5.0     1.021548      7
6.0     1.036890      7
7.0     0.984989      7
8.0     1.001939     11
9.0     0.995327     12
10.0    1.014764      8
11.0    1.018527      7
12.0    0.981550     13
13.0    1.022919      9
14.0    1.046168     10
15.0    1.035573      7
16.0    1.007711      4
17.0    1.043191      6
18.0    1.023015      7
19.0    0.978067      2
20.0    1.010624      6
21.0    0.953443      2
22.0    0.969769      2
23.0    0.998532      2
30.0    0.948997      2
0.522292993631 157 1.00951753701


/Users/sgcy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
